# FIT (Fast Import Tool)

The FIT imports appropriate data from NCBI GEO.

**Enter the series for import:**

In [1]:
series = "GSE7079"

### Libraries used

To install libraries, from R console use:
```R
install.packages("ggplot2")
```

For Bioconductor packages first get biocLite via
```R
source("https://bioconductor.org/biocLite.R")
```
Then install packages. Example:
```R
biocLite("GEOquery")
```

In [2]:
library(GEOquery);
library(affy);
library(gcrma);
library(RColorBrewer);
library(affyPLM);
library(simpleaffy);

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Welcome to Bioconductor

    

### Get/Create directories

Assumes this notebook is in notebook folder.

In [3]:
notebook_dir <- getwd()
main_dir <- dirname(notebook_dir)
gse_dir = file.path(main_dir,"gseRaw")
if (!dir.exists(gse_dir)) {
    dir.create(gse_dir)
}

### Load GEO data

In [4]:
gse <- getGEO(GEO = series, destdir = gse_dir)

Found 1 file(s)
GSE7079_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)
See spec(...) for full column specifications.
File stored at: 
/Users/terek/Documents/Github/GenClass-Stability/gseRaw/GPL85.soft


In [5]:
getGEOSuppFiles(GEO = series, makeDirectory = TRUE, baseDir = gse_dir)

,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
/Users/terek/Documents/Github/GenClass-Stability/gseRaw/GSE7079/GSE7079_RAW.tar,417187840,FALSE,644,2018-08-13 20:23:30,2018-08-13 20:23:30,2018-08-13 20:23:30,501,20,terek,staff


[1] "/Users/terek/Documents/Github"

In [29]:
gse <- getGEO(series)

Found 1 file(s)
GSE7079_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)
See spec(...) for full column specifications.
File stored at: 
/var/folders/qc/g4qqknrn1035gprs6yrmshwm0000gn/T//Rtmpb66U8a/GPL85.soft


In [30]:
show(gse)

$GSE7079_series_matrix.txt.gz
ExpressionSet (storageMode: lockedEnvironment)
assayData: 8799 features, 209 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: GSM170276 GSM170277 ... GSM170484 (209 total)
  varLabels: title geo_accession ... Tissue :ch1 (34 total)
  varMetadata: labelDescription
featureData
  featureNames: A01157cds_s_at A03913cds_s_at ... Z96106_at (8799
    total)
  fvarLabels: ID GB_ACC ... Gene Ontology Molecular Function (16 total)
  fvarMetadata: Column Description labelDescription
experimentData: use 'experimentData(object)'
Annotation: GPL85 

